<a href="https://colab.research.google.com/github/Riyanchaudhary/hackathon/blob/main/v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install ffmpeg
!pip install pydub


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [2]:
!pip install transformers sentencepiece speechrecognition pydub openai


In [3]:
import os
import speech_recognition as sr
from pydub import AudioSegment
from transformers import BartTokenizer, BartForConditionalGeneration



In [4]:
!pip install gradio



In [5]:
!pip install --upgrade gradio


In [ ]:
import os
import re
import gradio as gr
import speech_recognition as sr
from pydub import AudioSegment
from transformers import BartTokenizer, BartForConditionalGeneration

# Load summarization model
model_name = "philschmid/bart-large-cnn-samsum"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Function: Convert MP3 to WAV
def convert_to_wav(mp3_path):
    wav_path = mp3_path.replace(".mp3", ".wav")
    audio = AudioSegment.from_file(mp3_path, format="mp3")
    audio.export(wav_path, format="wav")
    return wav_path

# Function: Transcribe Audio
def transcribe_audio(audio_path):
    recognizer = sr.Recognizer()

    with sr.AudioFile(audio_path) as source:
        audio = recognizer.record(source)

    try:
        transcript = recognizer.recognize_google(audio)
        return transcript
    except Exception as e:
        return f"Error in transcription: {str(e)}"

# Function: Summarize Transcription
def summarize_meeting(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=1024)
    summary_ids = model.generate(inputs.input_ids, max_length=200, min_length=50, length_penalty=2.0, num_beams=4)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Gradio Wrapper Function
def process_audio(mp3_path):
    if not mp3_path.endswith(".mp3"):
        return "Error: Please upload an MP3 file!", ""

    # Convert MP3 to WAV
    wav_path = convert_to_wav(mp3_path)

    # Transcribe the WAV file
    transcription = transcribe_audio(wav_path)

    # Summarize the transcription
    summary = summarize_meeting(transcription)

    return transcription, summary

# Gradio Interface
iface = gr.Interface(
    fn=process_audio,
    inputs=gr.Audio(type="filepath"),
    outputs=[gr.Textbox(label="Transcription"), gr.Textbox(label="Summary")],
    title="🎤 Audio Meeting Summarizer",
    description="📌 Upload an MP3 file, and this tool will transcribe & summarize it.",
)

# Launch UI
iface.launch(share=True, debug=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   7%|7         | 115M/1.63G [00:00<?, ?B/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

* Running on public URL: https://c3e0c8a774e5cc25b6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
